Using sources:

https://huggingface.co/blog/fine-tune-vit

## Set up drive and libraries

In [30]:
# from google.colab import drive
import numpy as np
from datasets import load_dataset
import sys
import torch
# import timm
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
from src.datasets.imagenet1k import make_imagenet1k as asdf
from src.transforms import make_transforms
from src import helper
from src.models import vision_transformer
from datasets import Dataset, DatasetDict
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


Get CIFAR-10

In [ ]:
cifar10 = load_dataset("cifar10")

train_set = cifar10['train']
test_set = cifar10['test']

In [3]:
CHECKPOINT_FILE = './checkpoint/IN1K-vit.h.14-300e.pth.tar'

checkpoint = torch.load(CHECKPOINT_FILE, map_location=torch.device('cpu'))

/tmp/ipykernel_11247/1908512989.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CHECKPOINT_FILE, map_location=torch.device('cpu'))


In [ ]:

target_encoder_weights = checkpoint['target_encoder']
del checkpoint
import gc
gc.collect()


9

In [5]:
import copy
d = torch.device('cuda:0')
torch.cuda.set_device(d)
encoder, predictor = helper.init_model(
    device = d,
    patch_size=14,
    crop_size=224,
    pred_depth=12,
    pred_emb_dim=384,
    model_name='vit_huge')
target_encoder = copy.deepcopy(encoder)

INFO:root:VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14))
  )
  (blocks): ModuleList(
    (0-31): 32 x Block(
      (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1280, out_features=3840, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1280, out_features=1280, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=1280, out_features=5120, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
)


In [6]:
target_encoder_weights = {k.replace('module.', ''): v for k, v in target_encoder_weights.items()}
target_encoder.eval()
for p in target_encoder.parameters():
    p.requires_grad = False

msg = target_encoder.load_state_dict(target_encoder_weights)

In [15]:
transform = make_transforms(
    crop_size=224,
    crop_scale=[0.3, 1.0],
    gaussian_blur=False,
    horizontal_flip=False,
    color_distortion=False,
    color_jitter=0.0)

transform_with_resize = transforms.Compose([
  transforms.Resize((224, 224)),
  transform
  ])

INFO:root:making imagenet data transforms


In [16]:
class EncodeClassifyNet(nn.Module):
    def __init__(self, target_encoder):
        super().__init__()

        self.model = nn.Sequential(
            target_encoder,
            nn.AvgPool2d(kernel_size=4, stride=4),
            nn.Flatten(),
            nn.Linear(320 * 64, 10)
        )

    def forward(self, x):
        return self.model(x)

In [28]:
NN = EncodeClassifyNet(target_encoder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NN = NN.to(device)



## Below is mostly from ChatGPT. Should probably rewrite it

In [29]:
def evaluate_model(model, data_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for images, labels in data_loader:
            images, labels = images.cuda(), labels.cuda()

            # Forward pass
            outputs = model.forward(images)
            
            # Get predicted labels
            _, predicted = torch.max(outputs, 1)

            # Update totals
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


In [ ]:
# Example dataset wrapping (assuming train_set has an appropriate structure)
class CustomDataset(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        img = self.transform(item['img'])
        label = item['label']
        return img, label

In [31]:
test_dataset = CustomDataset(test_set, transform=transform)  # Assuming transform is defined
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

test_accuracy = evaluate_model(NN, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 8.35%


In [ ]:
# Define parameters
num_epochs = 1
batch_size = 32
learning_rate = 0.001

# Wrap your dataset and use DataLoader for batching
train_dataset = CustomDataset(train_set, transform=transform)  # Assuming transform is defined
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define model, loss function, and optimizer
model = NN.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


test_dataset = CustomDataset(test_set, transform=transform)  # Assuming transform is defined
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

test_accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    epoch_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        # Move data to GPU
        images, labels = images.cuda(), labels.cuda()

        # Forward pass
        outputs = model.forward(images)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate epoch loss
        epoch_loss += loss.item()

        # Print progress
        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}] completed with average loss: {epoch_loss / len(train_loader):.4f}")

print("Training complete!")


Epoch [1/1], Step [10/1094], Loss: 2.5746
Epoch [1/1], Step [20/1094], Loss: 1.5566
Epoch [1/1], Step [30/1094], Loss: 0.9655
Epoch [1/1], Step [40/1094], Loss: 0.9511
Epoch [1/1], Step [50/1094], Loss: 1.1058
Epoch [1/1], Step [60/1094], Loss: 0.6162
Epoch [1/1], Step [70/1094], Loss: 1.0499
Epoch [1/1], Step [80/1094], Loss: 0.6860
Epoch [1/1], Step [90/1094], Loss: 0.6967
Epoch [1/1], Step [100/1094], Loss: 1.1554
Epoch [1/1], Step [110/1094], Loss: 0.4802
Epoch [1/1], Step [120/1094], Loss: 0.9401
Epoch [1/1], Step [130/1094], Loss: 0.8504
Epoch [1/1], Step [140/1094], Loss: 0.6854
Epoch [1/1], Step [150/1094], Loss: 1.2742
Epoch [1/1], Step [160/1094], Loss: 0.5814
Epoch [1/1], Step [170/1094], Loss: 0.7537
Epoch [1/1], Step [180/1094], Loss: 0.8328
Epoch [1/1], Step [190/1094], Loss: 0.8534
Epoch [1/1], Step [200/1094], Loss: 0.9405
Epoch [1/1], Step [210/1094], Loss: 0.9304
Epoch [1/1], Step [220/1094], Loss: 1.1394
Epoch [1/1], Step [230/1094], Loss: 0.4194
Epoch [1/1], Step [2

In [26]:
test_dataset = CustomDataset(test_set, transform=transform)  # Assuming transform is defined
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

test_accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 81.50%
